In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import time

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')
x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')

In [3]:
num_classes = 10
n1 = 10
no_users = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [4]:
x_train = np.reshape(x_train, (x_train.shape[0], -1))/255
x_test = np.reshape(x_test, (x_test.shape[0], -1))/255

In [5]:
# Network parameters.
n_hidden_1 = 128 # 1st layer number of neurons.
n_hidden_2 = 64 # 2nd layer number of neurons.

In [6]:
class User:
    def __init__(self):
        self.weights = {
            'W1': tf.Variable(tf.random.truncated_normal([784,n_hidden_1], stddev=0.1)),
            'W2': tf.Variable(tf.random.truncated_normal([n_hidden_1,n_hidden_2], stddev=0.1)),
            'W3': tf.Variable(tf.random.truncated_normal([n_hidden_2,10], stddev=0.1)),
        }

        self.lambda_W1_l = tf.Variable(tf.zeros([784*n_hidden_1,1]))
        self.lambda_W2_l = tf.Variable(tf.zeros([n_hidden_1*n_hidden_2,1]))
        self.lambda_W3_l = tf.Variable(tf.zeros([n_hidden_2*10,1]))
        
        self.lambda_W1_r = tf.Variable(tf.zeros([784*n_hidden_1,1]))
        self.lambda_W2_r = tf.Variable(tf.zeros([n_hidden_1*n_hidden_2,1]))
        self.lambda_W3_r = tf.Variable(tf.zeros([n_hidden_2*10,1]))
        
        

    def neural_net(self, x):
        y1 = tf.nn.relu(tf.matmul(x, self.weights['W1']))
        y2 = tf.nn.relu(tf.matmul(y1, self.weights['W2']))
        ylogits = tf.matmul(y2, self.weights['W3'])
        return tf.nn.softmax(ylogits)

In [7]:
# mini-batch loss function.
def mini_batches(X, Y, mb_size = 100):

    m = X.shape[0]

    perm = list(np.random.permutation(m))
    #perm = perm_init[0:100]
    X_temp = X[perm,:]
    Y_temp = Y[perm,:].reshape((m, Y.shape[1]))
    
    X_r = tf.convert_to_tensor(X_temp[0:mb_size,:], dtype=np.float32)
    Y_r = tf.convert_to_tensor(Y_temp[0:mb_size,:], dtype=np.float32)
    return X_r,Y_r

In [8]:
# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy.
    return -tf.reduce_sum(y_true * tf.math.log(y_pred))

In [9]:
# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

In [10]:
# Optimization process. 
def run_optimization(x, y, user, user_l, user_r, rho, flag):
    # Wrap computation inside a GradientTape for automatic differentiation.
    #temp1_r = tf.reshape(user_r.weights['W1'],[784*n_hidden_1, 1])
    #temp2_r = tf.reshape(user_r.weights['W2'],[n_hidden_1*n_hidden_2, 1])
    #temp3_r = tf.reshape(user_r.weights['W3'],[10*n_hidden_2, 1])
    
    #temp1_l = tf.reshape(user_l.weights['W1'],[784*n_hidden_1, 1])
    #temp2_l = tf.reshape(user_l.weights['W2'],[n_hidden_1*n_hidden_2, 1])
    #temp3_l = tf.reshape(user_l.weights['W3'],[10*n_hidden_2, 1])
    
    with tf.GradientTape() as g:
        #g.watch(x)
        pred = user.neural_net(x)
        if (flag == 0):
            loss = cross_entropy(pred, y)\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W1'],[784*n_hidden_1, 1])-tf.reshape(user_r.weights['W1'],[784*n_hidden_1, 1])))\
            + tf.reduce_sum(tf.math.multiply(user.lambda_W1_r,tf.reshape(user.weights['W1'],[784*n_hidden_1, 1])-tf.reshape(user_r.weights['W1'],[784*n_hidden_1, 1])))\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W2'],[n_hidden_1*n_hidden_2, 1])-tf.reshape(user_r.weights['W2'],[n_hidden_1*n_hidden_2, 1]))) \
            + tf.reduce_sum(tf.math.multiply(user.lambda_W2_r,tf.reshape(user.weights['W2'],[n_hidden_1*n_hidden_2, 1])-tf.reshape(user_r.weights['W2'],[n_hidden_1*n_hidden_2, 1])))\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W3'],[10*n_hidden_2, 1])-tf.reshape(user_r.weights['W3'],[10*n_hidden_2, 1]))) \
            + tf.reduce_sum(tf.math.multiply(user.lambda_W3_r,tf.reshape(user.weights['W3'],[10*n_hidden_2, 1])-tf.reshape(user_r.weights['W3'],[10*n_hidden_2, 1])))

        elif (flag == 1):
            loss = cross_entropy(pred, y)\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W1'],[784*n_hidden_1, 1])-tf.reshape(user_l.weights['W1'],[784*n_hidden_1, 1])))\
            + tf.reduce_sum(tf.math.multiply(user.lambda_W1_l,-tf.reshape(user.weights['W1'],[784*n_hidden_1, 1])+tf.reshape(user_l.weights['W1'],[784*n_hidden_1, 1])))\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W2'],[n_hidden_1*n_hidden_2, 1])-tf.reshape(user_l.weights['W2'],[n_hidden_1*n_hidden_2, 1]))) \
            + tf.reduce_sum(tf.math.multiply(user.lambda_W2_l,-tf.reshape(user.weights['W2'],[n_hidden_1*n_hidden_2, 1])+tf.reshape(user_l.weights['W2'],[n_hidden_1*n_hidden_2, 1])))\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W3'],[10*n_hidden_2, 1])-tf.reshape(user_l.weights['W3'],[10*n_hidden_2, 1]))) \
            + tf.reduce_sum(tf.math.multiply(user.lambda_W3_l,-tf.reshape(user.weights['W3'],[10*n_hidden_2, 1])+tf.reshape(user_l.weights['W3'],[10*n_hidden_2, 1])))

        else:
            loss = cross_entropy(pred, y)\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W1'],[784*n_hidden_1, 1])-tf.reshape(user_r.weights['W1'],[784*n_hidden_1, 1])))\
            + tf.reduce_sum(tf.math.multiply(user.lambda_W1_r,tf.reshape(user.weights['W1'],[784*n_hidden_1, 1])-tf.reshape(user_r.weights['W1'],[784*n_hidden_1, 1])))\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W2'],[n_hidden_1*n_hidden_2, 1])-tf.reshape(user_r.weights['W2'],[n_hidden_1*n_hidden_2, 1]))) \
            + tf.reduce_sum(tf.math.multiply(user.lambda_W2_r,tf.reshape(user.weights['W2'],[n_hidden_1*n_hidden_2, 1])-tf.reshape(user_r.weights['W2'],[n_hidden_1*n_hidden_2, 1])))\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W3'],[10*n_hidden_2, 1])-tf.reshape(user_r.weights['W3'],[10*n_hidden_2, 1]))) \
            + tf.reduce_sum(tf.math.multiply(user.lambda_W3_r,tf.reshape(user.weights['W3'],[10*n_hidden_2, 1])-tf.reshape(user_r.weights['W3'],[10*n_hidden_2, 1])))
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W1'],[784*n_hidden_1, 1])-tf.reshape(user_l.weights['W1'],[784*n_hidden_1, 1])))\
            + tf.reduce_sum(tf.math.multiply(user.lambda_W1_l,-tf.reshape(user.weights['W1'],[784*n_hidden_1, 1])+tf.reshape(user_l.weights['W1'],[784*n_hidden_1, 1])))\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W2'],[n_hidden_1*n_hidden_2, 1])-tf.reshape(user_l.weights['W2'],[n_hidden_1*n_hidden_2, 1]))) \
            + tf.reduce_sum(tf.math.multiply(user.lambda_W2_l,-tf.reshape(user.weights['W2'],[n_hidden_1*n_hidden_2, 1])+tf.reshape(user_l.weights['W2'],[n_hidden_1*n_hidden_2, 1])))\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W3'],[10*n_hidden_2, 1])-tf.reshape(user_l.weights['W3'],[10*n_hidden_2, 1]))) \
            + tf.reduce_sum(tf.math.multiply(user.lambda_W3_l,-tf.reshape(user.weights['W3'],[10*n_hidden_2, 1])+tf.reshape(user_l.weights['W3'],[10*n_hidden_2, 1])))

        
    
    # Variables to update, i.e. trainable variables.
    trainable_variables = user.weights.values()
    
    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    #print(gradients)
    
    # Update W following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    return cross_entropy(pred, y)

In [11]:
users = [User() for i in range(no_users)]
rho = 20
optimizer = tf.optimizers.Adam(learning_rate=0.001)#0.001
#optimizer = tf.optimizers.SGD(learning_rate=0.001)
#central_modal = [tf.Variable(tf.random.truncated_normal([784*n_hidden_1,1], stddev=0.1)), tf.Variable(tf.random.truncated_normal([n_hidden_1*n_hidden_2,1], stddev=0.1)), tf.Variable(tf.random.truncated_normal([10*n_hidden_2,1], stddev=0.1))]

In [12]:
x_train_k = []
y_train_k = []
data_per_worker = int(x_train.shape[0]/no_users)
for i in range(no_users):
    first = i*data_per_worker
    last = first + data_per_worker
    x_train_k.append(x_train[first:last])
    y_train_k.append(y_train[first:last])

In [13]:
x_test = tf.convert_to_tensor(x_test, dtype=np.float32)
y_test = tf.convert_to_tensor(y_test, dtype=np.float32)

In [14]:
user0 = User()

In [15]:
import time

abs_weights_diff = []
abs_biases_diff = []
Train_Acc = []
Test_Acc = []
CrE_Train = []
CrE_Test = []


mb_size = 100
n_epochs = 400

compTime = np.zeros([n_epochs,1])

acc_train = np.zeros([n_epochs,1])
acc_test = np.zeros([n_epochs,1])
acc_test_max = np.zeros([n_epochs,1])
total_loss = np.zeros([n_epochs,1])
acc_test_user0 = np.zeros([n_epochs,1])
alpha=0.01#0.01
ii = 0
localIter = 10
for k in range(n_epochs):
        print(k)
        if k > 0:
            compTime[k] = compTime[k-1]
            
        batch_x = []
        batch_y = []
        for i in range(no_users):
            batch_xx , batch_yy = mini_batches(x_train_k[i],y_train_k[i],  mb_size)
            batch_x.append(batch_xx)
            batch_y.append(batch_yy)
        
                
        # Run the optimization to update W  
        #for i in range(no_users):
        for i in range(0,no_users,2):
            if(i==0):
                flag = 0
                t = time.time()
                for kk in range(localIter):
                    loss = run_optimization(batch_x[i], batch_y[i], users[i], users[i], users[i+1], rho, flag)
                elapsed = time.time() - t
                compTime[k]=compTime[k]+elapsed
            elif (i == no_users-1):
                flag = 1
                for kk in range(localIter):
                    loss = run_optimization(batch_x[i], batch_y[i], users[i], users[i-1], users[i], rho, flag)
            else:
                flag = 2
                for kk in range(localIter):
                    loss = run_optimization(batch_x[i], batch_y[i], users[i], users[i-1], users[i+1], rho, flag)
                    
            total_loss[k] = total_loss[k] + loss
            
        for i in range(1,no_users,2):
            if(i==0):
                flag = 0
                for kk in range(localIter):
                    loss = run_optimization(batch_x[i], batch_y[i], users[i], users[i], users[i+1], rho, flag)
            elif (i == no_users-1):
                flag = 1
                for kk in range(localIter):
                    loss = run_optimization(batch_x[i], batch_y[i], users[i], users[i-1], users[i], rho, flag)
            else:
                flag = 2
                for kk in range(localIter):
                    loss = run_optimization(batch_x[i], batch_y[i], users[i], users[i-1], users[i+1], rho, flag)
            
            
            
            total_loss[k] = total_loss[k] + loss
        
        total_loss[k] = total_loss[k]/no_users
        print(total_loss[k])
        
        primal_residual=0
        for i in range(no_users):
            W1 = tf.reshape(users[i].weights['W1'],[784*n_hidden_1, 1])
            W2 = tf.reshape(users[i].weights['W2'],[n_hidden_1*n_hidden_2, 1])
            W3 = tf.reshape(users[i].weights['W3'],[10*n_hidden_2, 1])
            if (i==0):
                W1_r = tf.reshape(users[i+1].weights['W1'],[784*n_hidden_1, 1])
                W2_r = tf.reshape(users[i+1].weights['W2'],[n_hidden_1*n_hidden_2, 1])
                W3_r = tf.reshape(users[i+1].weights['W3'],[10*n_hidden_2, 1])
                
                yy1 = (users[i].lambda_W1_r + alpha*rho*(W1 - W1_r)).numpy()
                yy2 = (users[i].lambda_W2_r + alpha*rho*(W2 - W2_r)).numpy()
                yy3 = (users[i].lambda_W3_r + alpha*rho*(W3 - W3_r)).numpy()
                
                users[i].lambda_W1_r.assign(yy1) 
                users[i].lambda_W2_r.assign(yy2) 
                users[i].lambda_W3_r.assign(yy3)
                
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W1 - W1_r)).numpy()
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W2 - W2_r)).numpy()
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W3 - W3_r)).numpy()
            
            elif (i==no_users-1):
                W1_l = tf.reshape(users[i-1].weights['W1'],[784*n_hidden_1, 1])
                W2_l = tf.reshape(users[i-1].weights['W2'],[n_hidden_1*n_hidden_2, 1])
                W3_l = tf.reshape(users[i-1].weights['W3'],[10*n_hidden_2, 1])
                
                yy1 = (users[i].lambda_W1_l - alpha*rho*(W1 - W1_l)).numpy()
                yy2 = (users[i].lambda_W2_l - alpha*rho*(W2 - W2_l)).numpy()
                yy3 = (users[i].lambda_W3_l - alpha*rho*(W3 - W3_l)).numpy()
                
                users[i].lambda_W1_l.assign(yy1) 
                users[i].lambda_W2_l.assign(yy2) 
                users[i].lambda_W3_l.assign(yy3)
                
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W1 - W1_l)).numpy()
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W2 - W2_l)).numpy()
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W3 - W3_l)).numpy()
            
            else:
                W1_r = tf.reshape(users[i+1].weights['W1'],[784*n_hidden_1, 1])
                W2_r = tf.reshape(users[i+1].weights['W2'],[n_hidden_1*n_hidden_2, 1])
                W3_r = tf.reshape(users[i+1].weights['W3'],[10*n_hidden_2, 1])
                
                W1_l = tf.reshape(users[i-1].weights['W1'],[784*n_hidden_1, 1])
                W2_l = tf.reshape(users[i-1].weights['W2'],[n_hidden_1*n_hidden_2, 1])
                W3_l = tf.reshape(users[i-1].weights['W3'],[10*n_hidden_2, 1])
                
                
                yy1 = (users[i].lambda_W1_r + alpha*rho*(W1 - W1_r)).numpy()
                yy2 = (users[i].lambda_W2_r + alpha*rho*(W2 - W2_r)).numpy()
                yy3 = (users[i].lambda_W3_r + alpha*rho*(W3 - W3_r)).numpy()
                
                yy4 = (users[i].lambda_W1_l - alpha*rho*(W1 - W1_l)).numpy()
                yy5 = (users[i].lambda_W2_l - alpha*rho*(W2 - W2_l)).numpy()
                yy6 = (users[i].lambda_W3_l - alpha*rho*(W3 - W3_l)).numpy()
                
                users[i].lambda_W1_r.assign(yy1) 
                users[i].lambda_W2_r.assign(yy2) 
                users[i].lambda_W3_r.assign(yy3)
            
                users[i].lambda_W1_l.assign(yy4) 
                users[i].lambda_W2_l.assign(yy5) 
                users[i].lambda_W3_l.assign(yy6)
                
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W1 - W1_r)).numpy()
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W2 - W2_r)).numpy()
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W3 - W3_r)).numpy()
                
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W1 - W1_l)).numpy()
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W2 - W2_l)).numpy()
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W3 - W3_l)).numpy()
                
            
            #users[i].lambda_W1.assign(yy1) 
            #users[i].lambda_W2.assign(yy2) 
            #users[i].lambda_W3.assign(yy3)
            
        print(primal_residual)
        
        
        train_acc = []
        test_acc = []
        for j in range(no_users):  
            train_pred = users[j].neural_net(batch_x[j])
            train_acc.append(accuracy(train_pred , batch_y[j]))
            test_pred = users[j].neural_net(x_test)
            test_acc.append(accuracy(test_pred , y_test))
            
        avgAcc_Train = np.mean(train_acc)
        avgAcc_Test = np.mean(test_acc)
        max_Test = np.max(test_acc)
        print(avgAcc_Train)
        print(avgAcc_Test)
        print(max_Test)
        print(compTime[k])
        acc_train[k] = avgAcc_Train
        acc_test[k] = avgAcc_Test
        acc_test_max[k] = max_Test
            
        #user0.weights['W1'] = tf.reshape(central_modal[0],[784,n_hidden_1])
        #user0.weights['W2'] = tf.reshape(central_modal[1],[n_hidden_1,n_hidden_2])
        #user0.weights['W3'] = tf.reshape(central_modal[2],[n_hidden_2, 10])
        
        #test_pred = user0.neural_net(x_test)
        #test_acc_user0 = accuracy(test_pred, y_test).numpy()
        #acc_test_user0[k]=test_acc_user0
        #print(test_acc_user0)
            

#np.save('loss_analog_4096_3.npy', total_loss)

#np.save('acc_test_digital_4096_user0_3.npy', acc_test_user0)

#np.save('loss_digital_40960_5.npy', total_loss)
#np.save('acc_train_digital_40960_5.npy', acc_train)
#np.save('acc_test_digital_40960_5.npy', acc_test)

0
[217.73195801]
23630.008875846863
0.306
0.20629999
0.2857
[0.28277493]
1
[204.37554932]
14458.394026756287
0.486
0.34044
0.5197
[0.45213413]
2
[191.8293457]
8378.309129238129
0.49
0.38983998
0.5136
[0.63154793]
3
[171.77983398]
5007.408725261688
0.59000003
0.46746
0.7199
[0.80591583]
4
[144.98615723]
3227.3154668807983
0.62
0.48813
0.7353
[0.97880387]
5
[117.90341797]
2286.186225295067
0.738
0.62477
0.8027
[1.14725089]
6
[88.53912354]
1768.8520829677582
0.80600005
0.67753
0.8354
[1.33288097]
7
[68.49708252]
1456.6455882489681
0.85199994
0.74540997
0.8461
[1.50426793]
8
[52.26917114]
1247.4553612172604
0.88699996
0.79312
0.8582
[1.67548203]
9
[43.25117187]
1089.4576110839844
0.90500003
0.79196
0.8345
[1.85165]
10
[40.03006897]
959.4253695607185
0.92399997
0.82381
0.8577
[2.0248251]
11
[36.90571899]
848.9660459160805
0.917
0.8287299
0.8719
[2.19609213]
12
[33.626651]
750.620081782341
0.936
0.8419601
0.8673
[2.36556005]
13
[29.44789734]
662.5237343907356
0.946
0.84714
0.8732
[2.53344417

0.995
0.93679
0.9472
[22.74564934]
114
[2.20021992]
46.80842175707221
0.99899995
0.93672
0.9414
[22.94283319]
115
[3.85789757]
48.56205835938454
0.9940001
0.9369
0.9448
[23.13861418]
116
[3.73287506]
49.80067418515682
0.996
0.9332101
0.9422
[23.33983231]
117
[2.70709953]
49.23527467995882
0.99799997
0.93569005
0.9439
[23.75188828]
118
[3.41192474]
50.822378508746624
0.996
0.93103
0.9452
[23.96339536]
119
[3.11661301]
54.3407401368022
0.99700004
0.92923003
0.9427
[24.15872025]
120
[2.9601614]
54.12069749832153
0.99700004
0.93429005
0.9426
[24.34597421]
121
[3.31604576]
53.46098639070988
0.995
0.9335799
0.9437
[24.53787422]
122
[2.40753975]
51.59995959699154
0.99899995
0.9361199
0.944
[24.76088524]
123
[3.93249435]
51.22491233050823
0.99399996
0.9353601
0.948
[24.96357536]
124
[3.1304697]
54.13533728942275
0.99799997
0.93271005
0.9435
[25.15136337]
125
[2.47558365]
50.44239918142557
0.99799997
0.94032
0.9445
[25.34379435]
126
[4.46213188]
53.0466964058578
0.995
0.93148994
0.9421
[25.6004

[3.37239151]
51.043652050197124
0.995
0.94447994
0.9566
[44.66417909]
225
[2.02834644]
55.78055642917752
0.99899995
0.93886995
0.948
[44.82628202]
226
[2.81708832]
55.86224803328514
0.99899995
0.9410299
0.9532
[44.98870611]
227
[1.59816418]
55.53050347045064
0.99799997
0.94537
0.9527
[45.15517402]
228
[1.55810194]
51.25957961939275
0.99899995
0.9471
0.9526
[45.32513809]
229
[3.03236885]
53.319843612611294
0.995
0.94202006
0.9528
[45.48643804]
230
[3.64948959]
54.934758234769106
0.99799997
0.94428
0.953
[45.65802407]
231
[1.17959223]
52.34524868056178
0.99899995
0.94769
0.9553
[45.82523513]
232
[2.28347073]
48.66662632673979
0.99700004
0.94846
0.9561
[45.98879409]
233
[3.12013874]
48.63256600499153
0.99899995
0.9445499
0.9538
[46.16210103]
234
[1.60325203]
50.54341256432235
0.99899995
0.94724005
0.9545
[46.32430696]
235
[2.35677109]
54.00311396829784
0.99700004
0.94609004
0.956
[46.49416995]
236
[1.53716431]
54.363920299336314
0.99799997
0.9452101
0.9582
[46.6703999]
237
[2.42854595]
56

[2.49525814]
54.39882067218423
0.996
0.94881
0.9584
[63.77611661]
334
[1.62368164]
56.258307468146086
0.9940001
0.94689
0.9597
[63.95060849]
335
[1.19859734]
59.217905377969146
1.0
0.94068
0.9613
[64.12568951]
336
[1.37370977]
57.44150183349848
0.99899995
0.94613
0.9606
[64.2887454]
337
[0.91312218]
55.26763051562011
0.99899995
0.95105
0.9613
[64.45451641]
338
[3.2227993]
61.751393876969814
0.99799997
0.94473
0.9609
[64.7089293]
339
[1.30165577]
61.21965132839978
0.99899995
0.94703007
0.9603
[64.96657729]
340
[1.78543606]
58.368591072037816
0.99700004
0.94521
0.9591
[65.12941337]
341
[1.73032913]
59.02312741521746
0.99899995
0.94498
0.9594
[65.30445623]
342
[0.84720936]
58.91858387738466
1.0
0.94234
0.9589
[65.4686172]
343
[1.55051184]
54.30709665454924
0.99899995
0.9494399
0.96
[65.6304183]
344
[1.93435974]
56.87930281460285
0.996
0.9422
0.9605
[65.80669117]
345
[1.21041565]
60.239072451367974
0.99799997
0.94337
0.9578
[65.96848416]
346
[2.22297115]
61.49078891798854
0.99799997
0.9443

In [16]:
#np.save('gadmm_dnn_train.npy', acc_train)
#np.save('gadmm_dnn_mean_test.npy', acc_test)
np.save('gadmm_dnn_max_test.npy', acc_test)
#np.save('gadmm_dnn_compTime.npy', compTime)